In [1]:
from ovcg import *
from fesol import *
from pyccx import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor
from mapper import generate_mapper

In [2]:
k=1
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

ccx_logoff = True
if ccx_logoff:
    import logging
    logger = logging.getLogger('PYCCX')
    logger.setLevel(logging.CRITICAL)


use_relax = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFPSP%i.pvd' % k)
fnodes = solverF.get_coordinates_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)

In [4]:
# solid solver
jobname = 'solid/solidD'
output = 'asc'
prob = Problem(jobname=jobname, output=output)
msolver = prob.get_solver()
msolver.initailize()
# since we know there is one step
# we stop b4 it
msolver.solve(stopb4step=1)
nlgeom = msolver.get_nlgeomsolver()
nlgeom.initialize()

# initialize our interface set
iset = nlgeom.get_set(
    set_name='Interface',
    set_type='sfelem',
    is_surface=True
)
nset = nlgeom.get_set(
    set_name='Inodes',
    set_type='node',
    is_surface=True
)

# retrieve geometry data
snodes = nset.coordinates()
scents = iset.extract_face_centers()

# create cht adapter
solverS = CHTAdapter(nlgeom=nlgeom)
solverS.add_interface(set_acc=nset, is_settable=True, itype=CHTAdapter.TEMP)
solverS.add_interface(set_acc=iset, is_settable=False, itype=CHTAdapter.FILM)

# now we are ready

In [5]:
Tf1 = 'Tf1'
Ts1 = 'Ts1'
hf = 'hf'
hs = 'hs'
Tf = 'Tf'
Ts = 'Ts'
Ff2Fs = {Tf:Ts}
Ts2Tf = {Ts1:Tf1,hs:hf}
_, s2f_mapper = generate_mapper(fnodes, scents, {}, Ts2Tf,r1=0.05,r2=0.2)
f2s_mapper, _ = generate_mapper(fnodes, snodes, Ff2Fs, {},r1=0.05,r2=0.2)

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=fnodes.shape[0], value=300.0)
abTempS = InterfaceData(size=scents.shape[0], value=310.0)
abTempS2F = InterfaceData(size=fnodes.shape[0], value=310.0)
tempF2S = InterfaceData(size=fnodes.shape[0], value=300.0)
tempS = InterfaceData(size=snodes.shape[0], value=310.0)
tempF = InterfaceData(size=fnodes.shape[0], value=300.0)
thetaT = InterfaceData(size=fnodes.shape[0], value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 300

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
hS = solverS['Interface', GET].get_sink_and_film()[1]

# map the coeff to fluid
s2f_mapper.put(hs, hS)
s2f_mapper.apply(hs, hf)
hS2F =s2f_mapper.get(hf)

CF1 = hS2F/(hS2F+hF)
CF2 = hF/(hS2F+hF)

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
dt = 0.05
t = 0.0
Nmax = 500
avg_pc_iterations = 0.0
flog = open('FPSP%i.log' % k, mode='w')
flog.write('Fluid pseudo Robin with solid pseudo Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FPSP%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup_state()
    
    # mapping
    s2f_mapper.put(Ts1, abTempS.curr)
    s2f_mapper.apply(Ts1, Tf1)
    abTempS2F.curr[:] = s2f_mapper.get(Tf1)
    if True:
        tempF.curr[:] = CF1*abTempS2F.curr + CF2*abTempF.curr
    else:
        # for first step, we use the thickness of plate as delta
        tmp = 400.0*np.ones(fnodes.shape[0])
        CFtmp1 = tmp/(tmp+hF)
        CFtmp2 = hF/(tmp+hF)
        tempF.curr[:] = CFtmp1*abTempS2F.curr+CFtmp2*abTempF.curr
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # compute wall temperature in fluid and map to solid
        # since we don't have method to project face center values
        # onto nodes, do this right now
        if True:
            tempF2S.curr[:] = CF1*abTempS2F.curr + CF2*abTempF.curr
        else:
            tempF2S.curr[:] = CFtmp1*abTempS2F.curr + CFtmp2*abTempF.curr
        
        # mapping
        f2s_mapper.put(Tf, tempF2S.curr)
        f2s_mapper.apply(Tf, Ts)
        tempS.curr[:] = f2s_mapper.get(Ts)
        # update solid flux interface
        solverS['Inodes', SET].set_temperatures(tempS.curr)
        
        # advance solid
        solverS.adjust_timesize(dt)
        solverS.increment()
        # retrieve solid interface temperature
        abTempS.curr[:] = solverS['Interface', GET].get_sink_and_film()[0]
        
        # mapping
        s2f_mapper.put(Ts1, abTempS.curr)
        s2f_mapper.apply(Ts1, Tf1)
        abTempS2F.curr[:] = s2f_mapper.get(Tf1)
        if True:
            tempF.curr[:] = CF1*abTempS2F.curr + CF2*abTempF.curr
        else:
            tempF.curr[:] = CFtmp1*abTempS2F.curr+CFtmp2*abTempF.curr
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverS.finish_increment()
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore_state()
            solverS.finish_increment()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        fluid_ofile.write(solverF, t)
flog.close()

nlgeom.finalize()
msolver.solve(skipsolve=True)
msolver.finalize()

step=0, pc_iterations=300.


KeyboardInterrupt: 

In [ ]:
theta=(solverF.get_temperature_interface(3,0,1)[0:100]-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFPSP%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

In [9]:
solverF.

array([0.0016808 , 0.00154658, 0.00148677, 0.00142722, 0.00137892,
       0.00134959, 0.00134105, 0.00135153, 0.00137802, 0.00141727,
       0.00146617, 0.00152213, 0.00158317, 0.0016476 , 0.00171356,
       0.00177867, 0.00184034, 0.00189685, 0.00194801, 0.00199478,
       0.00203804, 0.00207784, 0.00211367, 0.00214528, 0.00217305,
       0.00219776, 0.00221989, 0.00223942, 0.00225613, 0.00227014,
       0.0022821 , 0.00229265, 0.00230193, 0.0023095 , 0.00231488,
       0.00231812, 0.00231981, 0.00232051, 0.00232027, 0.00231866,
       0.00231536, 0.00231065, 0.00230522, 0.00229964, 0.00229386,
       0.00228748, 0.00228028, 0.00227264, 0.00226529, 0.00225865,
       0.00225254, 0.0022464 , 0.00223994, 0.00223336, 0.00222702,
       0.00222087, 0.00221421, 0.00220613, 0.00219605, 0.00218399,
       0.00217008, 0.00215409, 0.00213525, 0.0021129 , 0.00208715,
       0.00205906, 0.00203013, 0.00200163, 0.00197441, 0.00194919,
       0.00192689, 0.00190837, 0.00189388, 0.00188267, 0.00187